# Импорты

In [13]:
import numpy as np
from scipy.stats import norm
import networkx as nx

# Импорт данных из варианта
from base.task_option import *

# Класс сетевого анализа

In [14]:
class NetworkPlanning:
    def __init__(self):
        self.works = {}
        self.graph = nx.DiGraph()
        self.events = {}
        self.event_times = {}
        
    def add_work(self, name, dependencies, t_pes, t_ver, t_opt, _):
        """Добавление работы с её характеристиками"""
        self.works[name] = {
            'dependencies': dependencies,
            't_pes': t_pes,
            't_ver': t_ver,
            't_opt': t_opt
        }
    
    def calculate_three_param_model(self):
        """Расчет для трехпараметрической модели"""
        results = {}
        for name, work in self.works.items():
            t_oj = (work['t_pes'] + 4 * work['t_ver'] + work['t_opt']) / 6
            variance = ((work['t_pes'] - work['t_opt']) / 6) ** 2
            
            results[name] = {
                't_oj': round(t_oj),
                'variance': round(variance, 2)
            }
        return results
    
    def calculate_two_param_model(self):
        """Расчет для двухпараметрической модели"""
        results = {}
        for name, work in self.works.items():
            t_oj_star = (3 * work['t_pes'] + 2 * work['t_opt']) / 5
            variance = ((work['t_pes'] - work['t_opt']) / 6) ** 2
            
            results[name] = {
                't_oj_star': round(t_oj_star),
                'variance': round(variance, 2)
            }
        return results
    
    def build_correct_network(self, model_type='three_param'):
        """Правильное построение сетевого графика по методологии из файла"""
        self.graph.clear()
        self.events = {}
        
        if model_type == 'three_param':
            model_results = self.calculate_three_param_model()
            time_key = 't_oj'
        else:
            model_results = self.calculate_two_param_model()
            time_key = 't_oj_star'
        
        # Анализируем структуру зависимостей для правильного построения событий
        event_counter = 0
        work_to_end_event = {}
        
        # Начальное событие
        self.events[event_counter] = {'type': 'start'}
        start_event = event_counter
        event_counter += 1
        
        # Создаем события для работ без зависимостей
        for name, work in self.works.items():
            if work['dependencies'] == ['-']:
                end_event = event_counter
                self.events[end_event] = {'type': 'work_end', 'work': name}
                self.graph.add_edge(
                    start_event, end_event,
                    weight=model_results[name][time_key],
                    variance=model_results[name]['variance'],
                    name=name
                )
                work_to_end_event[name] = end_event
                event_counter += 1
        
        # Обрабатываем зависимые работы
        max_iterations = len(self.works) * 2  # Защита от бесконечного цикла
        iteration = 0
        
        while len(work_to_end_event) < len(self.works) and iteration < max_iterations:
            iteration += 1
            for name, work in self.works.items():
                if name in work_to_end_event:
                    continue  # Работа уже обработана
                
                # Проверяем, все ли зависимости обработаны
                dependencies_ready = True
                for dep in work['dependencies']:
                    if dep not in work_to_end_event:
                        dependencies_ready = False
                        break
                
                if dependencies_ready:
                    # Находим максимальное конечное событие среди зависимостей
                    max_dep_event = start_event
                    for dep in work['dependencies']:
                        max_dep_event = max(max_dep_event, work_to_end_event[dep])
                    
                    # Создаем конечное событие для текущей работы
                    end_event = event_counter
                    self.events[end_event] = {'type': 'work_end', 'work': name}
                    self.graph.add_edge(
                        max_dep_event, end_event,
                        weight=model_results[name][time_key],
                        variance=model_results[name]['variance'],
                        name=name
                    )
                    work_to_end_event[name] = end_event
                    event_counter += 1
        
        # Финальное событие - максимальное из всех конечных событий
        final_event = max(work_to_end_event.values()) if work_to_end_event else start_event
        self.events[final_event] = {'type': 'final'}
        
        return self.graph
    
    def calculate_event_times(self, model_type='three_param'):
        """Расчет ранних/поздних сроков событий"""
        graph = self.build_correct_network(model_type)
        
        if len(graph.nodes) == 0:
            return {'T_p': {}, 'T_n': {}, 'R': {}, 'critical_time': 0}
        
        # Прямой проход - расчет ранних сроков
        T_p = {0: 0}  # Ранние сроки, начальное событие = 0
        
        # Топологическая сортировка для прямого прохода
        try:
            topological_order = list(nx.topological_sort(graph))
        except nx.NetworkXError:
            # Граф имеет циклы, используем простой порядок
            topological_order = sorted(graph.nodes())
        
        for node in topological_order:
            if node not in T_p:
                T_p[node] = 0
            for pred in graph.predecessors(node):
                if pred in T_p:
                    edge_data = graph[pred][node]
                    edge_time = edge_data['weight']
                    T_p[node] = max(T_p[node], T_p[pred] + edge_time)
        
        # Обратный проход - расчет поздних сроков
        T_n = {}  # Поздние сроки
        final_node = max(topological_order)
        T_n[final_node] = T_p[final_node]  # Критическое время
        
        for node in reversed(topological_order):
            if node not in T_n:
                T_n[node] = T_p[final_node]
            for succ in graph.successors(node):
                if succ in T_n:
                    edge_data = graph[node][succ]
                    edge_time = edge_data['weight']
                    T_n[node] = min(T_n[node], T_n[succ] - edge_time)
        
        # Расчет резервов времени
        R = {node: T_n[node] - T_p[node] for node in topological_order}
        
        self.event_times = {
            'T_p': T_p,
            'T_n': T_n,
            'R': R,
            'critical_time': T_p[final_node]
        }
        
        return self.event_times
    
    def find_all_paths(self, model_type='three_param'):
        """Нахождение всех возможных путей в графе с учетом model_type"""
        graph = self.build_correct_network(model_type)
        
        if len(graph.nodes) == 0:
            return []
        
        start_node = 0
        end_node = max(graph.nodes())
        
        try:
            all_paths = list(nx.all_simple_paths(graph, start_node, end_node))
        except:
            # Если не удается найти все пути, используем упрощенный подход
            all_paths = [[start_node, end_node]] if graph.has_edge(start_node, end_node) else []
        
        paths_with_characteristics = []
        
        for path in all_paths:
            path_duration = 0
            path_variance = 0
            path_works = []
            
            for i in range(len(path) - 1):
                if graph.has_edge(path[i], path[i+1]):
                    edge_data = graph[path[i]][path[i+1]]
                    path_duration += edge_data['weight']
                    path_variance += edge_data['variance']
                    path_works.append(edge_data['name'])
            
            # Фильтруем пути, содержащие только реальные работы (без фиктивных)
            if path_works and not any(name.startswith('final_') for name in path_works):
                paths_with_characteristics.append({
                    'path': path,
                    'works': path_works,
                    'duration': path_duration,
                    'variance': path_variance,
                    'sigma': np.sqrt(path_variance) if path_variance > 0 else 0
                })
        
        return paths_with_characteristics
    
    def calculate_path_tension(self, model_type='three_param'):
        """Расчет коэффициентов напряженности путей с учетом model_type"""
        event_times = self.calculate_event_times(model_type)
        all_paths = self.find_all_paths(model_type)
        critical_time = event_times['critical_time']
        
        if critical_time == 0:
            return all_paths
        
        for path_info in all_paths:
            path_duration = path_info['duration']
            # Коэффициент напряженности
            tension = path_duration / critical_time
            path_info['tension'] = round(tension, 3)  # Более точное округление
            
            # Исправленная классификация по зонам напряженности
            if tension >= 0.99:  # Критический путь (учитываем погрешность округления)
                path_info['zone'] = 'critical'
            elif tension >= 0.6:
                path_info['zone'] = 'subcritical'
            else:
                path_info['zone'] = 'reserve'
        
        return all_paths
    
    def analyze_alternative_critical_paths(self, model_type='three_param'):
        """Анализ альтернативных критических путей"""
        all_paths = self.calculate_path_tension(model_type)
        critical_paths = [p for p in all_paths if p['zone'] == 'critical']
        
        if len(critical_paths) > 1:
            # Выбираем путь с наибольшим среднеквадратическим отклонением
            worst_path = max(critical_paths, key=lambda x: x['sigma'])
            
            analysis_result = {
                'multiple_critical_paths': True,
                'worst_path': worst_path,
                'all_critical_paths': critical_paths,
                'recommendation': f"Использовать путь с наибольшим σ = {worst_path['sigma']:.2f} для вероятностных расчетов"
            }
        else:
            analysis_result = {
                'multiple_critical_paths': False,
                'critical_path': critical_paths[0] if critical_paths else None
            }
        
        return analysis_result
    
    def find_critical_path(self, model_type='three_param'):
        """Нахождение критического пути"""
        all_paths = self.calculate_path_tension(model_type)
        
        # Ищем критические пути (напряженность ≈ 1.0)
        critical_paths = [p for p in all_paths if p['zone'] == 'critical']
        
        if not critical_paths:
            # Если нет путей с напряженностью 1.0, берем самый длинный
            critical_paths = sorted(all_paths, key=lambda x: x['duration'], reverse=True)
        
        if critical_paths:
            critical_path = critical_paths[0]
            return {
                'critical_time': critical_path['duration'],
                'critical_works': critical_path['works'],
                'total_variance': critical_path['variance'],
                'sigma': critical_path['sigma'],
                'path_events': critical_path['path'],
                'tension': critical_path['tension']
            }
        else:
            return None
    
    def calculate_probability(self, T_dir, model_type='three_param'):
        """Расчет вероятности выполнения проекта к заданному сроку"""
        critical_path = self.find_critical_path(model_type)
        if not critical_path:
            return None
        
        T_kr = critical_path['critical_time']
        sigma = critical_path['sigma']
        
        # Избегаем деления на ноль
        if sigma == 0:
            return 1.0 if T_dir >= T_kr else 0.0
        
        z = (T_dir - T_kr) / sigma
        probability = norm.cdf(z)
        
        return probability
    
    def guaranteed_interval(self, P=0.9973, model_type='three_param'):
        """Интервал гарантированного времени выполнения"""
        critical_path = self.find_critical_path(model_type)
        if not critical_path:
            return None
        
        T_kr = critical_path['critical_time']
        sigma = critical_path['sigma']
        
        if sigma == 0:
            return (T_kr, T_kr)
        
        if P == 0.9973:
            delta = 3 * sigma
        else:
            z = norm.ppf((1 + P) / 2)
            delta = z * sigma
        
        return (T_kr - delta, T_kr + delta)
    
    def max_duration_with_reliability(self, gamma=0.95, model_type='three_param'):
        """Максимальный срок выполнения с заданной надежностью"""
        critical_path = self.find_critical_path(model_type)
        if not critical_path:
            return None
        
        T_kr = critical_path['critical_time']
        sigma = critical_path['sigma']
        
        if sigma == 0:
            return T_kr
        
        z_gamma = norm.ppf(gamma)
        T_max = T_kr + z_gamma * sigma
        
        return T_max
    
    def get_work_reserves(self, model_type='three_param'):
        """Расчет резервов времени для работ"""
        event_times = self.calculate_event_times(model_type)
        graph = self.build_correct_network(model_type)
        
        work_reserves = {}
        
        for u, v, data in graph.edges(data=True):
            if 'name' not in data:
                continue
                
            work_name = data['name']
            t_ij = data['weight']
            
            # Проверяем, что события существуют в расчетах
            if u in event_times['T_p'] and v in event_times['T_n']:
                # Полный резерв времени работы
                full_reserve = event_times['T_n'][v] - event_times['T_p'][u] - t_ij
                
                work_reserves[work_name] = {
                    'full_reserve': full_reserve,
                    'early_start': event_times['T_p'][u],
                    'late_start': event_times['T_n'][v] - t_ij,
                    'early_finish': event_times['T_p'][u] + t_ij,
                    'late_finish': event_times['T_n'][v],
                    'is_critical': abs(full_reserve) < 0.001  # Работа критическая, если резерв ≈ 0
                }
        
        return work_reserves
    
    def get_critical_works_from_events(self, model_type='three_param'):
        """Определение критических работ на основе резервов событий"""
        event_times = self.calculate_event_times(model_type)
        work_reserves = self.get_work_reserves(model_type)
        
        critical_works = []
        for work_name, reserves in work_reserves.items():
            if reserves['is_critical']:
                critical_works.append(work_name)
        
        return critical_works

# Основные расчеты

In [15]:
# Создаем экземпляр класса
np_solver = NetworkPlanning()

for work in works_data:
    np_solver.add_work(*work)

print("=== ТРЕХПАРАМЕТРИЧЕСКАЯ МОДЕЛЬ ===")

# Расчет ожидаемого времени и дисперсии
three_param = np_solver.calculate_three_param_model()
print("\nОжидаемое время и дисперсия (трехпараметрическая модель):")
for work, values in three_param.items():
    print(f"{work}: t_ож = {values['t_oj']}, D(x) = {values['variance']}")

# Расчет времен событий и резервов
event_times_3param = np_solver.calculate_event_times('three_param')
print(f"\nКритическое время (через события): {event_times_3param['critical_time']} дней")

# Анализ всех путей и напряженности
all_paths_3param = np_solver.calculate_path_tension()
critical_paths_3param = [p for p in all_paths_3param if p['zone'] == 'critical']

print(f"\nНайдено критических путей: {len(critical_paths_3param)}")
for i, path in enumerate(critical_paths_3param[:3]):  # Показываем первые 3
    print(f"Путь {i+1}: {path['works']} (длительность: {path['duration']} дней, напряженность: {path['tension']})")

# Критический путь (обновленный метод)
critical_3param = np_solver.find_critical_path('three_param')
if critical_3param:
    print(f"\nОсновной критический путь: {critical_3param['critical_works']}")
    print(f"Ожидаемое время выполнения: {critical_3param['critical_time']} дней")
    print(f"Дисперсия критического пути: {critical_3param['total_variance']:.2f}")
    print(f"Среднеквадратическое отклонение: {critical_3param['sigma']:.2f}")
    
    # Вероятность выполнения к сроку
    prob_3param = np_solver.calculate_probability(T_dir, 'three_param')
    print(f"\nВероятность выполнения к сроку {T_dir} дней: {prob_3param:.2%}")
    
    # Интервал гарантированного выполнения
    interval_3param = np_solver.guaranteed_interval(0.9973, 'three_param')
    print(f"Интервал гарантированного выполнения (P=0.9973): {interval_3param[0]:.1f} - {interval_3param[1]:.1f} дней")
    
    # Максимальный срок с надежностью
    max_duration_3param = np_solver.max_duration_with_reliability(gamma, 'three_param')
    print(f"Максимальный срок с надежностью {gamma}: {max_duration_3param:.1f} дней")

# Анализ альтернативных критических путей
alt_analysis_3param = np_solver.analyze_alternative_critical_paths()
if alt_analysis_3param['multiple_critical_paths']:
    print(f"\nОбнаружено несколько критических путей!")
    print(f"Рекомендация: {alt_analysis_3param['recommendation']}")

print("\n" + "="*50)
print("=== ДВУХПАРАМЕТРИЧЕСКАЯ МОДЕЛЬ ===")

# Расчет для двухпараметрической модели
two_param = np_solver.calculate_two_param_model()
print("\nОжидаемое время и дисперсия (двухпараметрическая модель):")
for work, values in two_param.items():
    print(f"{work}: t*_ож = {values['t_oj_star']}, D(x) = {values['variance']}")

# Расчет времен событий и резервов для двухпараметрической модели
event_times_2param = np_solver.calculate_event_times('two_param')
print(f"\nКритическое время (через события): {event_times_2param['critical_time']} дней")

# Анализ всех путей и напряженности
all_paths_2param = np_solver.calculate_path_tension()
critical_paths_2param = [p for p in all_paths_2param if p['zone'] == 'critical']

print(f"\nНайдено критических путей: {len(critical_paths_2param)}")
for i, path in enumerate(critical_paths_2param[:3]):
    print(f"Путь {i+1}: {path['works']} (длительность: {path['duration']} дней, напряженность: {path['tension']})")

# Критический путь
critical_2param = np_solver.find_critical_path('two_param')
if critical_2param:
    print(f"\nОсновной критический путь: {critical_2param['critical_works']}")
    print(f"Ожидаемое время выполнения: {critical_2param['critical_time']} дней")
    print(f"Дисперсия критического пути: {critical_2param['total_variance']:.2f}")
    print(f"Среднеквадратическое отклонение: {critical_2param['sigma']:.2f}")
    
    # Вероятность выполнения к сроку
    prob_2param = np_solver.calculate_probability(T_dir, 'two_param')
    print(f"\nВероятность выполнения к сроку {T_dir} дней: {prob_2param:.2%}")
    
    # Интервал гарантированного выполнения
    interval_2param = np_solver.guaranteed_interval(0.9973, 'two_param')
    print(f"Интервал гарантированного выполнения (P=0.9973): {interval_2param[0]:.1f} - {interval_2param[1]:.1f} дней")
    
    # Максимальный срок с надежностью
    max_duration_2param = np_solver.max_duration_with_reliability(gamma, 'two_param')
    print(f"Максимальный срок с надежностью {gamma}: {max_duration_2param:.1f} дней")

# Анализ альтернативных критических путей
alt_analysis_2param = np_solver.analyze_alternative_critical_paths()
if alt_analysis_2param['multiple_critical_paths']:
    print(f"\nОбнаружено несколько критических путей!")
    print(f"Рекомендация: {alt_analysis_2param['recommendation']}")

print("\n" + "="*50)
print("=== СРАВНЕНИЕ РЕЗУЛЬТАТОВ ===")
if critical_3param and critical_2param:
    time_diff = critical_2param['critical_time'] - critical_3param['critical_time']
    prob_diff = prob_2param - prob_3param
    
    print(f"Разница в ожидаемом времени: {time_diff} дней")
    print(f"Разница в вероятности выполнения: {prob_diff:.2%}")
    
    # Анализ различий в критических путях
    if critical_3param['critical_works'] != critical_2param['critical_works']:
        print(f"Различие в критических путях:")
        print(f"  3-параметрическая: {critical_3param['critical_works']}")
        print(f"  2-параметрическая: {critical_2param['critical_works']}")

# Дополнительная информация о резервах времени
print("\n" + "="*50)
print("=== ДОПОЛНИТЕЛЬНЫЙ АНАЛИЗ ===")

# Резервы времени для работ (трехпараметрическая модель)
work_reserves = np_solver.get_work_reserves('three_param')
print("\nРезервы времени работ (трехпараметрическая модель):")
print("Работа | Полный резерв | Ранний старт | Поздний старт")
print("-" * 55)
for work, reserves in list(work_reserves.items())[:6]:  # Показываем первые 6
    print(f"{work:6} | {reserves['full_reserve']:13.1f} | {reserves['early_start']:12.1f} | {reserves['late_start']:12.1f}")

# Анализ подкритических путей
subcritical_paths = [p for p in all_paths_3param if p['zone'] == 'subcritical']
if subcritical_paths:
    print(f"\nПодкритические пути (напряженность 0.6-0.8): {len(subcritical_paths)}")
    for path in subcritical_paths[:2]:  # Показываем первые 2
        print(f"  {path['works']} (напряженность: {path['tension']})")

=== ТРЕХПАРАМЕТРИЧЕСКАЯ МОДЕЛЬ ===

Ожидаемое время и дисперсия (трехпараметрическая модель):
b1: t_ож = 6, D(x) = 0.69
b2: t_ож = 4, D(x) = 0.44
b3: t_ож = 10, D(x) = 1.36
b4: t_ож = 2, D(x) = 0.25
b5: t_ож = 4, D(x) = 0.25
b6: t_ож = 8, D(x) = 1.0
b7: t_ож = 3, D(x) = 0.69
b8: t_ож = 4, D(x) = 1.36
b9: t_ож = 6, D(x) = 1.36
b10: t_ож = 5, D(x) = 0.44
b11: t_ож = 9, D(x) = 1.0

Критическое время (через события): 20 дней

Найдено критических путей: 1
Путь 1: ['b2', 'b7', 'b8', 'b11'] (длительность: 20 дней, напряженность: 1.0)

Основной критический путь: ['b2', 'b7', 'b8', 'b11']
Ожидаемое время выполнения: 20 дней
Дисперсия критического пути: 3.49
Среднеквадратическое отклонение: 1.87

Вероятность выполнения к сроку 26 дней: 99.93%
Интервал гарантированного выполнения (P=0.9973): 14.4 - 25.6 дней
Максимальный срок с надежностью 0.95: 23.1 дней

=== ДВУХПАРАМЕТРИЧЕСКАЯ МОДЕЛЬ ===

Ожидаемое время и дисперсия (двухпараметрическая модель):
b1: t*_ож = 6, D(x) = 0.69
b2: t*_ож = 4, D(x) =